In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
data = pd.read_csv("pune.csv");

In [3]:
data.columns

Index(['date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour',
       'uvIndex', 'uvIndex.1', 'moon_illumination', 'moonrise', 'moonset',
       'sunrise', 'sunset', 'DewPointC', 'FeelsLikeC', 'HeatIndexC',
       'WindChillC', 'WindGustKmph', 'cloudcover', 'humidity', 'precipMM',
       'pressure', 'tempC', 'visibility', 'winddirDegree', 'windspeedKmph'],
      dtype='object')

In [4]:
data.drop("date_time",axis=1)
#store the temp in new var
weather_y = data.pop('tempC')
weather_x = data.drop("date_time",axis=1)

In [5]:
weather_x.head()

,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,moonrise,moonset,sunrise,...,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,windspeedKmph
0,31,13,0.0,11.0,6,1,31,10:17 AM,10:16 PM,07:07 AM,...,18,18,7,0,50,0.0,1013,10,59,3
1,31,13,0.0,11.0,6,1,31,10:17 AM,10:16 PM,07:07 AM,...,18,18,9,0,47,0.0,1013,10,57,4
2,31,13,0.0,11.0,6,1,31,10:17 AM,10:16 PM,07:07 AM,...,18,18,10,0,44,0.0,1013,10,55,5
3,31,13,0.0,11.0,6,1,31,10:17 AM,10:16 PM,07:07 AM,...,18,18,12,0,41,0.0,1013,10,54,6
4,31,13,0.0,11.0,6,1,31,10:17 AM,10:16 PM,07:07 AM,...,20,20,11,1,38,0.0,1014,10,68,6


In [6]:
moonrise_time = weather_x.pop('moonrise')
moonset_time = weather_x.pop('moonset')
sunrise_time = weather_x.pop('sunrise')
sunset_time = weather_x.pop('sunset')
new_moonrise_format = []
new_sunrise_format = []
new_moonset_format = []
new_sunset_format = []

In [7]:
for i in range(96432):
    if moonrise_time[i] == 'No moonrise' or moonset_time[i]== 'No moonset':
        moonrise_time[i] = moonrise_time[i-1]
        moonset_time[i] = moonset_time[i-1]
        

In [8]:
for i in range (96432):
    new_moonrise_time = moonrise_time[i]
    new_sunrise_time = sunrise_time[i]
    new_moonset_time = moonset_time[i]
    new_sunset_time = sunset_time[i]
   
    (mrt,mrdt) = new_moonrise_time.split(' ')
    (srt,srdt) = new_sunrise_time.split(' ')
    (mst,msdt) = new_moonset_time.split(' ')
    (sst,ssdt) = new_sunset_time.split(' ')

    (mrh,mrm) = mrt.split(':')
    (srh,srm) = srt.split(':')
    (msh,msm) = mst.split(':')
    (ssh,ssm) = sst.split(':')
    
    modified_moonrise_time = float(mrh) + (float(mrm)/60)
    modified_sunrise_time = float(srh) + (float(srm)/60)
    modified_moonset_time = float(msh) + (float(msm)/60) + 12
    modified_sunset_time = float(ssh) + (float(ssm)/60) + 12
    
    new_moonrise_format.append(modified_moonrise_time)
    new_sunrise_format.append(modified_sunrise_time)
    new_moonset_format.append(modified_moonset_time)
    new_sunset_format.append(modified_sunset_time)
  

In [9]:
mr_dataframe=pd.DataFrame(new_moonrise_format, columns=['moonrise'])
sr_dataframe=pd.DataFrame(new_sunrise_format, columns=['sunrise'])
ms_dataframe=pd.DataFrame(new_moonset_format, columns=['moonset'])
ss_dataframe=pd.DataFrame(new_sunset_format, columns=['sunset'])

In [10]:
ms_dataframe.shape

(96432, 1)

In [11]:
ss_dataframe.shape

(96432, 1)

In [12]:

weather_x = pd.concat((weather_x,mr_dataframe,sr_dataframe,ms_dataframe,ss_dataframe),axis=1)
weather_x.head()

,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,DewPointC,FeelsLikeC,HeatIndexC,...,humidity,precipMM,pressure,visibility,winddirDegree,windspeedKmph,moonrise,sunrise,moonset,sunset
0,31,13,0.0,11.0,6,1,31,7,18,18,...,50,0.0,1013,10,59,3,10.283333,7.116667,22.266667,18.15
1,31,13,0.0,11.0,6,1,31,6,18,18,...,47,0.0,1013,10,57,4,10.283333,7.116667,22.266667,18.15
2,31,13,0.0,11.0,6,1,31,6,18,18,...,44,0.0,1013,10,55,5,10.283333,7.116667,22.266667,18.15
3,31,13,0.0,11.0,6,1,31,5,18,18,...,41,0.0,1013,10,54,6,10.283333,7.116667,22.266667,18.15
4,31,13,0.0,11.0,6,1,31,5,20,20,...,38,0.0,1014,10,68,6,10.283333,7.116667,22.266667,18.15


In [13]:
weather_y.head()

0    13
1    14
2    14
3    15
4    16
Name: tempC, dtype: int64

In [14]:
## x is ref to input ans y as output 
x_train,x_test,y_train,y_test = train_test_split(weather_x,weather_y,test_size = 0.2,random_state = 123) 

In [15]:
x_train.head()

,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,DewPointC,FeelsLikeC,HeatIndexC,...,humidity,precipMM,pressure,visibility,winddirDegree,windspeedKmph,moonrise,sunrise,moonset,sunset
42545,29,19,0.0,8.7,6,6,33,17,26,26,...,65,0.0,1011,10,54,1,11.150000,6.616667,22.816667,17.983333
95978,29,19,0.0,11.4,6,1,79,13,20,21,...,64,0.0,1014,10,257,4,8.133333,6.966667,20.766667,18.000000
20824,36,24,0.0,13.0,7,8,100,15,34,34,...,35,0.0,1007,10,297,12,8.150000,6.000000,18.483333,19.033333
64516,36,24,0.0,11.6,8,1,36,18,28,28,...,60,0.0,1009,10,279,7,10.316667,6.033333,23.583333,19.000000
29492,36,22,0.0,11.6,7,1,44,19,27,27,...,68,0.0,1008,10,295,13,12.866667,6.016667,24.833333,19.016667


In [16]:
x_train.shape

(77145, 23)

In [17]:
y_train.head()

42545    24
95978    20
20824    33
64516    26
29492    25
Name: tempC, dtype: int64

In [18]:
y_train.shape

(77145,)

In [19]:
#x_test.head()
x_test.head()

,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,DewPointC,FeelsLikeC,HeatIndexC,...,humidity,precipMM,pressure,visibility,winddirDegree,windspeedKmph,moonrise,sunrise,moonset,sunset
60572,30,20,0.0,8.7,6,1,71,13,22,25,...,56,0.0,1015,10,98,6,9.400000,6.816667,21.766667,17.933333
62416,31,15,0.0,11.3,6,7,42,8,27,27,...,31,0.0,1011,10,120,5,10.600000,7.050000,23.416667,18.566667
90411,41,29,0.0,11.6,9,1,43,12,29,29,...,34,0.0,1007,10,316,11,12.500000,6.166667,23.850000,18.900000
253,31,20,0.0,11.1,6,7,100,10,29,29,...,31,0.0,1013,10,167,4,6.616667,7.166667,19.200000,18.250000
19354,38,16,0.0,11.6,7,9,94,0,41,41,...,8,0.0,1009,10,90,5,5.416667,6.683333,17.183333,18.750000


In [20]:
x_test.shape

(19287, 23)

In [21]:
x_train.columns

Index(['maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex',
       'uvIndex.1', 'moon_illumination', 'DewPointC', 'FeelsLikeC',
       'HeatIndexC', 'WindChillC', 'WindGustKmph', 'cloudcover', 'humidity',
       'precipMM', 'pressure', 'visibility', 'winddirDegree', 'windspeedKmph',
       'moonrise', 'sunrise', 'moonset', 'sunset'],
      dtype='object')

In [ ]:
model_param = {
    'LinearRegression':{
        'model':LinearRegression(),
         'param':{
           
        }
    },
        'DecisionTreeRegressor':{
        'model':DecisionTreeRegressor(),
        'param':{
           
        }
    },
        'RandomForestRegressor':{
        'model':RandomForestRegressor(),
        'param':{
            'max_depth':[10,25,50,75],
        }
    }
}

In [ ]:
scores =[]
for model_name, mp in model_param.items():
    model_selection = GridSearchCV(estimator = mp['model'],param_grid = mp['param'], cv = 5, return_train_score = False)
    model_selection.fit(x_train,y_train)
    scores.append({
        'model': model_name,
        'best_score': model_selection.best_score_,
        'best_params': model_selection.best_params_
    })

In [ ]:
model_score = pd.DataFrame(scores,columns=['model','best_score','best_params'])
model_score

In [22]:
model =LinearRegression()

In [23]:
model.fit(x_train,y_train)


LinearRegression()

In [24]:
pred = model.predict(x_test)

In [25]:
pred.shape

(19287,)

In [26]:
y_test.shape

(19287,)

In [27]:
#calculating RMSE
np.sqrt(np.mean((pred - y_test)**2))

0.7997731534908586

In [28]:
#polynomial regression
poly = PolynomialFeatures(degree = 3)
x_poly = poly.fit_transform(x_train)
lin = LinearRegression()
lin.fit(x_poly,y_train)

LinearRegression()

In [29]:
pred2 = lin.predict(poly.fit_transform(x_test))

In [30]:
np.sqrt(np.mean((pred2 - y_test)**2))

0.5026990466979245

In [31]:
#decision tree regression
deceReg = DecisionTreeRegressor(random_state = 0)
deceReg.fit(x_train,y_train)

DecisionTreeRegressor(random_state=0)

In [32]:
pred3 = deceReg.predict(x_test)

In [33]:
np.sqrt(np.mean((pred3 - y_test)**2))

0.6166596250538593

In [34]:
regr = RandomForestRegressor(max_depth=50, random_state=0,n_estimators=100)
regr.fit(x_train,y_train)

RandomForestRegressor(max_depth=50, random_state=0)

In [35]:
pred4 = regr.predict(x_test)
print(pred4)
np.sqrt(np.mean((pred4 - y_test)**2))
#print(accuracy_score( y_test, pred4))

[21.6  27.02 28.61 ... 17.21 22.97 25.04]


0.4497437150441888